# Laboratorio No. 4 - Deep Learning

In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM # type: ignore

In [34]:
data = pd.read_csv('data.csv')
data.head()

,Month,Sunspots
0,1749-01,58.0
1,1749-02,62.6
2,1749-03,70.0
3,1749-04,55.7
4,1749-05,85.0


In [35]:
data['Month'] = pd.to_datetime(data['Month'])
data['Month'] = data['Month'].dt.year + data['Month'].dt.month / 12.0

### Modelo Feed Forward

In [36]:
X = data[['Month']].values
y = data['Sunspots'].values

In [37]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [38]:
model_ffnn = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

c:\Users\50242\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
model_ffnn.compile(optimizer='adam', loss='mean_squared_error')

In [40]:
model_ffnn.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 4236.5371 - val_loss: 2430.5930
Epoch 2/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2034.5524 - val_loss: 1825.9680
Epoch 3/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2006.0070 - val_loss: 1810.4850
Epoch 4/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2076.2217 - val_loss: 1801.0981
Epoch 5/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1985.7596 - val_loss: 1796.4369
Epoch 6/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2097.2393 - val_loss: 1793.9933
Epoch 7/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1986.4020 - val_loss: 1797.9647
Epoch 8/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1888.7317 - val_loss: 1809.7258
Epoch 9/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1948.9136 - val_loss: 1792.6858
Epoch 10/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1918.2205 - val_loss: 1793.0149
Epoch 11/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1893.2786 - val_loss: 1792.8192
Epoch 12

In [41]:
loss = model_ffnn.evaluate(X_test, y_test)
print(f'Loss modelo Feed Forward: {loss}')

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1685.5144 
Loss modelo Feed Forward: 1761.7608642578125


### Modelo RNN

In [42]:
sequence_length = 12
X_rnn = []
y_rnn = []

In [43]:
for i in range(len(X_scaled) - sequence_length):
    X_rnn.append(X_scaled[i:i+sequence_length])
    y_rnn.append(y[i+sequence_length])

X_rnn = np.array(X_rnn)
y_rnn = np.array(y_rnn)

In [44]:
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_rnn, y_rnn, test_size=0.2, random_state=42)

In [45]:
model_rnn = Sequential([
    LSTM(128, return_sequences=True, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    LSTM(64, return_sequences=True, activation='relu'),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])

c:\Users\50242\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [46]:
model_rnn.compile(optimizer='adam', loss='mean_squared_error')

In [47]:
model_rnn.fit(X_train_rnn, y_train_rnn, epochs=50, batch_size=32, validation_data=(X_test_rnn, y_test_rnn))

Epoch 1/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 3828.6589 - val_loss: 1989.5885
Epoch 2/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1958.3059 - val_loss: 1824.9293
Epoch 3/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1703.2501 - val_loss: 1851.9945
Epoch 4/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1837.0105 - val_loss: 1820.8582
Epoch 5/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1763.8877 - val_loss: 1841.9648
Epoch 6/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1837.8540 - val_loss: 1923.9257
Epoch 7/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1808.3712 - val_loss: 1838.3353
Epoch 8/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1794.9824 - val_loss: 1807.7437
Epoch 9/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1867.9280 - val_loss: 1813.3970
Epoch 10/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1908.8517 - val_loss: 1933.1608
Epoch 11/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1809.7491 - val_loss: 1769.74

In [48]:
loss_rnn = model_rnn.evaluate(X_test_rnn, y_test_rnn)
print(f'Loss modelo RNN: {loss_rnn}')

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1546.7444
Loss modelo RNN: 1666.087158203125
